# 리뷰 추천시스템

In [1]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.8 MB/s 
     |████████████████████████████████| 453 kB 57.8 MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import warnings
import re
warnings.filterwarnings(action='ignore')

## 데이터

In [4]:
df = pd.read_csv('/content/drive/MyDrive/데이터/빅페 부산식당/망고플레이트1.csv')
df.head()

,리뷰,맛,식당명,가고싶다,전체평점,주소,음식종류
0,\n 부산 최고의 웨이팅 핫플서울토박이로 살다가 간만에 부산을 놀러...,맛있다,톤쇼우,"1,580",4.7,부산시 수영구 민락동 181-20,까스 요리
1,\n 제 목숨을 바쳐도 좋어요.충청도 사는 소녀는 톤쇼우를 먹으러 ...,맛있다,톤쇼우,"1,580",4.7,부산시 수영구 민락동 181-20,까스 요리
2,\n 부산가서 무슨 돈까스 먹냐는 약간의 공격을 당했지만후기보니 너...,맛있다,톤쇼우,"1,580",4.7,부산시 수영구 민락동 181-20,까스 요리
3,\n 태어나서 먹어본 돈까스 중에 가장 맛있습니다가게 앞에 뚱하니 ...,맛있다,톤쇼우,"1,580",4.7,부산시 수영구 민락동 181-20,까스 요리
4,\n 캬 사진만으로도 너무나 가고싶었지만 웨이팅이 엄두가 안났던 톤...,맛있다,톤쇼우,"1,580",4.7,부산시 수영구 민락동 181-20,까스 요리


In [5]:
# 중복 삭제
df = df.drop_duplicates(['리뷰'])
df = df.reset_index()

In [6]:
df.tail()

,index,리뷰,맛,식당명,가고싶다,전체평점,주소,음식종류
990,1036,\n 카페 나폴리\n,괜찮다,카페인차저,52,3.9,부산시 수영구 민락동 179-11 202호,카페 / 디저트
991,1037,\n 탁 트인 광안리 뷰가 매력적인 에스프레소바. 저녁에 와서 음주...,맛있다,카페인차저,52,3.9,부산시 수영구 민락동 179-11 202호,카페 / 디저트
992,1038,\n 광안리 앞의 바다뷰 카페. 창문이 있는줄 알았는데 알고보니 없...,괜찮다,카페인차저,52,3.9,부산시 수영구 민락동 179-11 202호,카페 / 디저트
993,1039,\n 카페라떼 - 5500너티라떼 - 6000광안리 뷰가 너무 이뻐...,맛있다,카페인차저,52,3.9,부산시 수영구 민락동 179-11 202호,카페 / 디저트
994,1040,\n Cafe squino 2.8 / Latte gone nuts ...,맛있다,카페인차저,52,3.9,부산시 수영구 민락동 179-11 202호,카페 / 디저트


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   995 non-null    int64  
 1   리뷰      995 non-null    object 
 2   맛       995 non-null    object 
 3   식당명     995 non-null    object 
 4   가고싶다    995 non-null    object 
 5   전체평점    995 non-null    float64
 6   주소      995 non-null    object 
 7   음식종류    995 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 62.3+ KB


In [8]:
df['식당명'].value_counts()

톤쇼우            10
거대곰탕           10
속씨원한대구탕        10
블랙업커피          10
삼삼횟집           10
               ..
할매집원조복국         5
카페인차저           5
대쿠이             4
시그니엘부산 더라운지     4
합천일류돼지국밥        4
Name: 식당명, Length: 191, dtype: int64

In [9]:
# 식당별 리뷰합치기
agg = df.groupby(['식당명'])['리뷰'].agg({'unique'})
agg

,unique
식당명,
1984나폴리,[\n 위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\...
303화덕,"[\n 그냥 그냥 맛있는건 아니구요\n , \n ..."
Cafe de 220VOLT,[\n 커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이...
가온밀면,"[\n 해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹..."
거대갈비,"[\n 부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집..."
...,...
호랑이젤라떡,[\n 기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 ...
호찐빵,"[\n 음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라....."
화국반점,[\n 약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느...


In [10]:
agg = agg.reset_index()
agg

,식당명,unique
0,1984나폴리,[\n 위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\...
1,303화덕,"[\n 그냥 그냥 맛있는건 아니구요\n , \n ..."
2,Cafe de 220VOLT,[\n 커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이...
3,가온밀면,"[\n 해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹..."
4,거대갈비,"[\n 부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집..."
...,...,...
186,호랑이젤라떡,[\n 기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 ...
187,호찐빵,"[\n 음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라....."
188,화국반점,[\n 약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느...
189,희와제과,[\n 부산의 대표 빵집 희와제과♥원래는 가성비 맛집이라는 타이틀이...


In [11]:
# 리스트를 string으로
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [12]:
agg['String']=agg['unique'].apply(listToString)
agg

,식당명,unique,String
0,1984나폴리,[\n 위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\...,위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\n \n ...
1,303화덕,"[\n 그냥 그냥 맛있는건 아니구요\n , \n ...",그냥 그냥 맛있는건 아니구요\n \n 화덕피자는 첨이...
2,Cafe de 220VOLT,[\n 커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이...,커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이는 많은 부품과 머신들~...
3,가온밀면,"[\n 해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹...","해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹은 밀면 중에 가장 청결..."
4,거대갈비,"[\n 부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집...","부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집 중 한 곳이지요!! 와..."
...,...,...,...
186,호랑이젤라떡,[\n 기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 ...,기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 맛이 제일 괜찮았고젤라또...
187,호찐빵,"[\n 음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라.....","음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라.. 비가 생각보다 많이 왔..."
188,화국반점,[\n 약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느...,약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느낌의 탕수육이에요. 자극...
189,희와제과,[\n 부산의 대표 빵집 희와제과♥원래는 가성비 맛집이라는 타이틀이...,"부산의 대표 빵집 희와제과♥원래는 가성비 맛집이라는 타이틀이 있었는데, 오랜만에 방..."


In [13]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 ]', '', text)

In [14]:
agg['text']=agg['String'].apply(alpha_num)
agg

,식당명,unique,String,text
0,1984나폴리,[\n 위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\...,위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\n \n ...,위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와 ...
1,303화덕,"[\n 그냥 그냥 맛있는건 아니구요\n , \n ...",그냥 그냥 맛있는건 아니구요\n \n 화덕피자는 첨이...,그냥 그냥 맛있는건 아니구요 화덕피자는 첨이예요 인...
2,Cafe de 220VOLT,[\n 커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이...,커피(?)원두(?)머신(?)회사인 것 같았다.전문적으로 보이는 많은 부품과 머신들~...,커피원두머신회사인 것 같았다전문적으로 보이는 많은 부품과 머신들바 공간이 여유로워 ...
3,가온밀면,"[\n 해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹...","해운대역 근처에 위치한 '가온밀면'- 밀면, 만두부산에서 먹은 밀면 중에 가장 청결...",해운대역 근처에 위치한 가온밀면 밀면 만두부산에서 먹은 밀면 중에 가장 청결하고 맛...
4,거대갈비,"[\n 부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집...","부산 해운대에 위치한 ""거대갈비"". 해운대에서 유명한 고기집 중 한 곳이지요!! 와...",부산 해운대에 위치한 거대갈비 해운대에서 유명한 고기집 중 한 곳이지요 와 우리나라...
...,...,...,...,...
186,호랑이젤라떡,[\n 기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 ...,기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 맛이 제일 괜찮았고젤라또...,기본 맛과 해운대 맛은 22년 8월 기준 없어진듯피스타치오 맛이 제일 괜찮았고젤라또...
187,호찐빵,"[\n 음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라.....","음.. 찐빵맛집으로 소문난 호빵집 방문,가는날이 장날이라.. 비가 생각보다 많이 왔...",음 찐빵맛집으로 소문난 호빵집 방문가는날이 장날이라 비가 생각보다 많이 왔고 그래서...
188,화국반점,[\n 약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느...,약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느낌의 탕수육이에요. 자극...,약간은 특이한 맛의 탕수육인데 바삭한 느낌 보다는 촉촉한 느낌의 탕수육이에요 자극적...
189,희와제과,[\n 부산의 대표 빵집 희와제과♥원래는 가성비 맛집이라는 타이틀이...,"부산의 대표 빵집 희와제과♥원래는 가성비 맛집이라는 타이틀이 있었는데, 오랜만에 방...",부산의 대표 빵집 희와제과원래는 가성비 맛집이라는 타이틀이 있었는데 오랜만에 방문했...


In [15]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-tz0ggpt5
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-tz0ggpt5
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=35575b868c142e7e1aece31c62931f922587e3c9073961ed015ed24608e89923
  Stored in directory: /tmp/pip-ephem-wheel-cache-vmfw6bac/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
spacing = Spacing()

spaced_corpus = []
for sent in df['review']:
    spaced_corpus.append(spacing(sent))

df['text'] = spaced_corpus
df.tail(3)

https://wikidocs.net/92961

In [24]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [35]:
for i in range(len(agg)):
  try:
    result_train = spell_checker.check(agg['text'][i])
    agg['text'][i] = result_train.as_dict()['checked']
  except:
    pass

In [30]:
# 한국어 맞춤법 교정
from hanspell import spell_checker

spell_corpus = []
for sent in agg['text']:
  spelled_sent = spell_checker.check(sent)
  hanspell_sent = spelled_sent.checked
  spell_corpus.append(hanspell_sent)

agg['clean_text'] = spell_corpus
agg.head(2)

,식당명,unique,String,text,clean_text
0,1984나폴리,[\n 위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\...,위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와\n \n ...,위치가 골목 안이라 분위기가 새롭고 하몽피자 새롭다 새로와 ...,
1,303화덕,"[\n 그냥 그냥 맛있는건 아니구요\n , \n ...",그냥 그냥 맛있는건 아니구요\n \n 화덕피자는 첨이...,그냥 그냥 맛있는건 아니구요 화덕피자는 첨이예요 인...,


왜 안되지

In [54]:
agg.to_csv('/content/drive/MyDrive/데이터/빅페 부산식당/review.csv', index = False)

##  모델 : Doc2Vec 

In [55]:
df1 = pd.read_csv('/content/drive/MyDrive/데이터/빅페 부산식당/review.csv')

In [ ]:
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [56]:
from konlpy.tag import Mecab
from tqdm import tqdm
from gensim.models.doc2vec import TaggedDocument
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df1.iterrows(), total=len(df1)):
  text = row['text']
  tag = row['식당명']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print(len(tagged_corpus_list)) # 문서의 수

100%|██████████| 191/191 [00:01<00:00, 159.76it/s]

191


In [57]:
len(tagged_corpus_list) # 문서의 수

191

In [58]:
tagged_corpus_list[0]

TaggedDocument(words=['위치', '가', '골목', '안', '이', '라', '분위기', '가', '새롭', '고', '하몽', '피자', '새롭', '다', '새로', '와', '여긴', '생긴', '지', '정말', '오래', '됐', '지만', '변함', '없이', '맛있', '는', '거', '같', '아요', '사실', '은', '이쪽', '라인', '이', '가게', '가', '정말', '자주', '바뀌', '는데', '이렇게', '오래', '장사', '를', '하', '는', '것', '만', '으로', '도', '맛', '은', '보장', '되', '었', '다고', '생각', '합니다', '맛있', '는', '화덕', '피자', '에', '맥주', '한잔', '하', '면', '정말', '최고', '의', '조합', '같', '아요', '평소', '에', '정말', '궁금', '했', '던', '1984', '나폴리', '화덕', '피자', '가', '너무', '먹', '고', '싶', '고', '금주', '상태', '인', '지금', '술', '먹', '고', '싶', '은', '친구', '끌', '고', '갔', '는데', '만족', '만족', '종류', '가', '다양', '해서', '엄청', '고민', '하', '다가', '픽', '한', '것', '파스타', '는', '약간', '매콤', '함', '이', '있', '어서', '느끼', '하', '지', '않', '게', '호로록', '먹', '을', '수', '잇', '었', '고', '화덕', '피자', '는', '도우', '가', '뭔가', '조금', '다르', '단', '느낌', '화덕', '피자', '의', '고소', '함', '이나', '뭔가', '건조', '한', '느낌', '이', '매력', '인데', '여긴', '약간', '아주', '야야야', '간', '기름', '이', '느껴져서', '사알짝', '바싹', '느낌', '도', '있', '었', '다

**Doc2Vec**
https://radimrehurek.com/gensim/models/doc2vec.html


**vector_size** (int, optional) – Dimensionality of the feature vectors

**alpha** (float, optional) – The initial learning rate.

**min_alpha** (float, optional) – Learning rate will linearly drop to min_alpha as training progresses.

**workers** (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines)(multi cpu).

**window** (int, optional) – The maximum distance between the current and predicted word within a sentence.

**negative** (int, optional) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.(default 5)

In [59]:
from gensim.models import doc2vec
model = doc2vec.Doc2Vec(vector_size=300, 
                        alpha=0.025,
                        min_alpha=0.025,
                        workers=8,
                        window=8,
                        negative = 10)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

Tag Size: 191 / 

## 결과

In [60]:
# 10개 추천
similar_doc = model.docvecs.most_similar('호찐빵')
print(similar_doc)

[('마가만두', 0.4616379737854004), ('비앤씨', 0.38011640310287476), ('고래사', 0.3662528395652771), ('남천동 보성녹차 팥빙수', 0.3621383011341095), ('전포명가떡집 (휴업중)', 0.3452860116958618), ('흰여울비치', 0.3438278138637543), ('노홍만두', 0.3420170247554779), ('봉샌드', 0.33881527185440063), ('딤타오', 0.33691710233688354), ('해운대소문난암소갈비집', 0.3214009404182434)]


In [61]:
similar_doc = model.docvecs.most_similar('가온밀면')
print(similar_doc)

[('본가제일면가', 0.5191744565963745), ('해운대밀면', 0.511214554309845), ('대성밀냉면', 0.5052481293678284), ('원조밀면', 0.4964008331298828), ('원조부산밀면', 0.4677327871322632), ('국제밀면', 0.46408554911613464), ('내호냉면', 0.46064314246177673), ('노홍만두', 0.4412866234779358), ('공원칼국수', 0.41909927129745483), ('해운대31cm해물칼국수', 0.40066802501678467)]


전처리 추가하고 모델 파라미터 튜토리얼 확인하기